In [31]:
import torch
from x_transformers import XTransformer, TransformerWrapper, Encoder, Decoder
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import librosa
from playsound import playsound
from vector_quantize_pytorch import VectorQuantize
import torchaudio

In [46]:
filename = 'D:\\musicnet\\musicnet\\train_data\\1727.wav'

waveform, sample_rate = torchaudio.load(filename, normalize=True)
resample_rate = 3000

transformed = torchaudio.transforms
transformed = transformed.Resample(sample_rate, resample_rate)
resampled_waveform = transformed(waveform)[:,:512]
print(resampled_waveform, resampled_waveform.shape)

tensor([[-6.2255e-08,  1.6568e-07, -2.0146e-06, -2.5406e-07,  1.0626e-07,
         -5.1383e-08,  2.1609e-08, -5.7241e-09,  1.0875e-10,  0.0000e+00,
          0.0000e+00,  3.1706e-09, -2.3491e-08,  7.3499e-08, -1.8817e-07,
          4.5257e-07, -1.7268e-06, -6.2831e-07, -8.7892e-07, -1.3882e-06,
         -1.5320e-07, -1.7778e-06, -8.1579e-07, -1.7652e-06,  1.0746e-06,
         -1.7143e-06,  2.3247e-06,  3.9560e-06, -7.7178e-06, -2.3971e-06,
         -1.2847e-06, -8.0998e-08,  4.0612e-07, -1.3357e-06, -1.0618e-06,
         -5.0726e-07, -1.8547e-06,  2.1916e-06, -2.0063e-06, -1.6240e-06,
         -5.2815e-07,  5.0857e-07, -1.1383e-06, -1.2304e-06, -3.3296e-06,
          1.4462e-06,  7.0002e-06, -1.8543e-06, -6.3503e-07,  2.2166e-06,
          1.3640e-06,  3.6757e-07, -3.4094e-06, -3.2288e-06,  4.2208e-06,
         -2.2841e-06, -4.4643e-07, -1.3056e-06,  8.2650e-07, -5.6773e-06,
         -2.1909e-06,  5.2651e-07, -2.4907e-06,  3.9803e-06, -1.1796e-06,
         -2.1508e-06, -1.9369e-06,  1.

0.0

In [12]:
model = XTransformer(
    dim = 512,
    enc_num_tokens = 256,
    enc_depth = 6,
    enc_heads = 8,
    enc_max_seq_len = 1024,
    dec_num_tokens = 256,
    dec_depth = 6,
    dec_heads = 8,
    dec_max_seq_len = 1024,
    tie_token_emb = True      
)

src = torch.randint(0, 1, (1, 1024))
tgt = torch.randint(0, 1, (1, 1024))


criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    optimizer.zero_grad()
    yhat = model(src, tgt) # (1, 1024, 512)
    yhat2 = model(src, tgt)
    loss = criterion(yhat, yhat2)
    loss.backward()
    optimizer.step()

In [67]:
encoder = TransformerWrapper(
    num_tokens = 2000,
    max_seq_len = 512,
    attn_layers = Encoder(
        dim = 512,
        depth = 12,
        heads = 8
    )
)

x = torch.randint(0, 256, (1, 1024))
# mask = torch.randint(0, 1, x.shape).bool()
print(torch.min(resampled_waveform, axis=1), torch.max(resampled_waveform, axis=1)) 
wv = np.abs(resampled_waveform * 100000).int()
print(wv)
# print(initial_melody, x)
encoding = encoder(wv)
print(encoding.shape)

torch.return_types.min(
values=tensor([-0.0026]),
indices=tensor([469])) torch.return_types.max(
values=tensor([0.0028]),
indices=tensor([499]))
tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   1,   2,  15,  47,  44,  50,  40,  42,  31,  24,  10,
          20,  15,   3,   3,  13,  12,   0,  18,  15,  18,  33,  31,  15,  10,
          27,  44,  45,  59,  62,  39,  48,  42,  24,  23,  20,  39,  42,  20,
          12,   4,   2,  20,  21,   5,  17,   0,   7,  16,  27,  47,  33,   6,
           4,  10,  23,  39,   3,  29,  32,  36,  57,  56,  45,  19,  35,  37,
          26,  49,  52,  34,   7,  29,  35,  45,  40,  38,  24,  

In [72]:
decoder = TransformerWrapper(
    num_tokens = 2000,
    max_seq_len = 512,
    attn_layers = Decoder(
        dim = 512,
        depth = 12,
        heads = 8
    )
)

x = torch.rand((1, 1024))

decoder(x)

tensor([[0.6011, 0.4332, 0.4064,  ..., 0.5424, 0.9954, 0.1035]])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [52]:
vq = VectorQuantize(
    dim = 512,
    codebook_size = 256,
    codebook_dim = 32,
    use_cosine_sim = True#  paper proposes setting this to 32 or as low as 8 to increase codebook usage
)

x = torch.randn(1, 2048, 512)
quantized, indices, commit_loss = vq(x)